In [1]:
import os
import yaml
from getpass import getpass

from navigator_helpers import NL2PythonTaskSuite, GretelLLMSuite
from navigator_helpers.tasks.text_to_code.utils import display_nl2code_sample
from navigator_helpers.tasks.prompt_templates.template_suite import load_prompt_template_suite

In [2]:
gretel_api_key = os.environ.get("GRETEL_PROD_API_KEY") or getpass("Enter your Gretel API key: ")

llm_config = yaml.safe_load(f"""\
- model_name: gretelai-gpt-llama3-1-8b
  litellm_params:
    model: gretelai/gpt-llama3-1-8b
    api_key: {gretel_api_key}
    api_base: https://api.gretel.ai
  tags:
  - open_license
  - nl
  - code
  - judge
"""
)

llm_suite = GretelLLMSuite(llm_config=llm_config)

2024-09-24 10:55:17.900 - INFO - 🦜 Initializing LLM suite
2024-09-24 10:55:17.901 - INFO - 📖 Natural language LLM: gretelai-gpt-llama3-1-8b
2024-09-24 10:55:17.901 - INFO - 💻 Code LLM: gretelai-gpt-llama3-1-8b
2024-09-24 10:55:17.902 - INFO - ⚖️ Judge LLM: gretelai-gpt-llama3-1-8b


In [3]:
tasks = NL2PythonTaskSuite(llm_suite)

In [4]:
record = tasks.create_record(
    domain="Physics", 
    topic="Particle Physics", 
    complexity="Novice: Basic syntax, variables, and data types", 
    llm_as_a_judge=False, 
    syntax_validation=False
    semantic_validation=False
)

In [5]:
display_nl2code_sample(record)

                                                  Contextual Tags                                                  
┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Domain  ┃ Topic            ┃ Complexity                              ┃ Suggested Packages                       ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Physics │ Particle Physics │ Novice: Basic syntax, variables, and    │ scikit-learn, pandas, matplotlib, scipy, │
│         │                  │ data types                              │ numpy, astropy                           │
└─────────┴──────────────────┴─────────────────────────────────────────┴──────────────────────────────────────────┘

╭───────────────────────────────────────── Natural Language Instruction ──────────────────────────────────────────╮
│ Write a Python script that asks the user for the mass and charge of an electron and then calculates and         │
│ displays its momentum using the formula momentum = mass * velocity, assuming the velocity of an electron is     │
│ approximately 2.2 x 10^8 m/s.                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── Code ──────────────────────────────────────────────────────╮
│ import numpy as np                                                                                              │
│                                                                                                                 │
│ mass = float(input("Enter the mass of an electron in kg: "))                                                    │
│ charge = float(input("Enter the charge of an electron in C: "))                                                 │
│ velocity = 2.2e8  # m/s                                                                                         │
│                                                                                                                 │
│ momentum = mass * velocity                                                                                      │
│ print(f"The momentum of an electron is {momentum} kg m/s")                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [6]:
tags = tasks.generate_contextual_tags(
    num_domains=3,
    num_topics_per_domain=5,
    num_complexity_levels=4,
)

2024-09-24 10:55:22.103 - INFO - 🏷️ Generating domains
2024-09-24 10:55:23.209 - INFO - 🏷️ Generating topics for each domain
2024-09-24 10:55:26.994 - INFO - 🏷️ Generating levels of Python complexity


In [7]:
tags

ContextualTags({
    "domain_and_topics": {
        "Data Science Research Institutes": [
            "Machine Learning Algorithms",
            "Data Mining Techniques",
            "Predictive Analytics Tools",
            "Data Visualization Software",
            "Statistical Modeling Methods"
        ],
        "Financial Services Firms": [
            "Accounting and Bookkeeping",
            "Risk Management Systems",
            "Portfolio Management Tools",
            "Investment Portfolio Analysis",
            "Financial Reporting Software"
        ],
        "Automotive Technology Companies": [
            "Automotive Software Development",
            "Vehicle Operating Systems",
            "Autonomous Vehicle Technology",
            "Electric Vehicle Charging Systems",
            "Connected Car Platforms"
        ]
    },
    "complexity_levels": [
        "Beginner: Basic syntax and data types",
        "Intermediate: Functions and control structures",
        "Advan

In [8]:
prompt_templates = load_prompt_template_suite("python")

prompt_templates

PromptTemplateSuite(
    domains: ('num_domains',)
    topics_from_domains: ('num_topics', 'domain')
    complexity: ('num_levels',)
    python_natural_language: ('nl_type_description', 'nl_type', 'complexity', 'nl_type', 'domain', 'topic')
    python_code_generation: ('python_natural_language', 'complexity', 'domain', 'topic', 'suggested_packages')
    python_suggested_packages: ('domain', 'max_dependencies', 'domain')
)

In [9]:
print(prompt_templates.domains(num_domains=3))

Create a list of 3 unique industries where you expect to find software engineers who code in Python. 

### Instructions:
    * Do not use abbreviations.
    * Keep each industry name to 1-5 words, preferring concise names.
    * List the industries in a valid JSON array.

